In [1]:
import tensorflow as tf
import numpy as np
import os, time, random
from utils import str2bool, get_logger, get_entity
from data import random_embedding,tag2label
import datapreprocessing as dp
from model import BiLSTM_CRF

C:\Users\Ian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

In [3]:
## get char embeddings
word2id = dp.loadVocab('word2id.pkl')
type(word2id)

vocab_size: 1460


dict

In [4]:
embeddings = random_embedding(word2id, 300)
# embeddings = np.load('pretrain_embedding.npy')
type(embeddings),embeddings.shape

(numpy.ndarray, (1460, 300))

In [6]:
## paths setting
paths = {}
timestamp = str(int(time.time()))
output_path = os.path.join('.', 'data_path'+"_save", timestamp)
if not os.path.exists(output_path): os.makedirs(output_path)
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path
if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix
result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path
if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path
# get_logger(log_path).info(str(args))
get_logger(log_path).info('BiLSTM-CRF for Chinese NER task')

BiLSTM-CRF for Chinese NER task


In [5]:
## read corpus and get training data
train_data = dp.readCorpus('D:\\Desktop\\产品\\辅诊\\已标n\\corpus')

In [10]:
## hyperparameters-tuning, split train/dev
# 打乱数据
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(train_data)))
shuffle_indices

array([3244,  176, 1631, ...,  527, 3197, 1289])

In [14]:
train_data_shuffled = []
for i in shuffle_indices:
    train_data_shuffled.append(train_data[i])

In [15]:
# 数据集切分为两部分
test_sample_index = -1 * int(0.1 * float(len(train_data)))

dev_data = train_data_shuffled[test_sample_index:]; dev_size = len(dev_data)
train_data = train_data_shuffled[:test_sample_index]; train_size = len(train_data)
print("train data: {0}\ndev data: {1}".format(train_size, dev_size))

train data: 3297
dev data: 366


In [17]:
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths=paths, config=config)
model.build_graph()
model.train(train=train_data, dev=dev_data)  # use test_data as the dev_data to see overfitting phenomena
np.save('pretrain_embedding.npy',model.embeddings)

C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2018-07-04 11:20:15 epoch 1, step 1, loss: 89.72, global_step: 1


2018-07-04 11:20:15 epoch 1, step 52, loss: 17.54, global_step: 52
===========validation / test===========


2018-07-04 11:23:16 epoch 2, step 1, loss: 27.0, global_step: 53


2018-07-04 11:23:16 epoch 2, step 52, loss: 27.08, global_step: 104
===========validation / test===========


...

2018-07-04 13:12:58 epoch 38, step 52, loss: 1.959, global_step: 1976
===========validation / test===========


2018-07-04 13:16:11 epoch 39, step 1, loss: 2.159, global_step: 1977


2018-07-04 13:16:11 epoch 39, step 52, loss: 1.253, global_step: 2028
===========validation / test===========


2018-07-04 13:19:24 epoch 40, step 1, loss: 1.485, global_step: 2029


2018-07-04 13:19:24 epoch 40, step 52, loss: 1.703, global_step: 2080
===========validation / test===========


## 以前的训练结果

In [8]:
model.train(train=train_data, dev=dev_data)

C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


train data: 1862


2018-06-20 10:59:41 epoch 1, step 1, loss: 127.1, global_step: 1


2018-06-20 10:59:41 epoch 1, step 30, loss: 44.92, global_step: 30
===========validation / test===========


2018-06-20 11:01:53 epoch 2, step 1, loss: 51.96, global_step: 31


...

===========validation / test===========


2018-06-20 12:19:45 epoch 39, step 1, loss: 3.273, global_step: 1141


2018-06-20 12:19:45 epoch 39, step 30, loss: 2.888, global_step: 1170
===========validation / test===========


2018-06-20 12:22:01 epoch 40, step 1, loss: 2.022, global_step: 1171


2018-06-20 12:22:01 epoch 40, step 30, loss: 1.259, global_step: 1200
===========validation / test===========
